# Import

In [343]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib import rc
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import matplotlib.pyplot as plt
import re
import glob
import os
from pathlib import Path

# Read csv files

In [344]:
submission = pd.read_csv('../../data/sample_submission.csv')
holidays = pd.read_csv('../../data/holidays_2023_2025.csv')
train = pd.read_csv('../../data/train/train.csv')
# os.getcwd()
terms = pd.read_csv('../solar_term_2023_2025.csv')

# Feature about date
1. year
2. month
3. day
4. weekday
5. is_holiday
6. is_sandwich
7. is_before_holiday
8. is_after_holiday
9. is_weekend

In [345]:
# 1) 날짜 범위
dates = pd.date_range(start="2023-01-01", end="2025-05-31", freq="D")

# 연, 월, 일 col 생성
train['영업일자'] = pd.to_datetime(train['영업일자'])
train['time_idx'] = (train['영업일자'] - train['영업일자'].min()).dt.days
train['year'] = train['영업일자'].dt.year
train['month'] = train['영업일자'].dt.month
train['day'] = train['영업일자'].dt.day
# 영업장, 메뉴명 col 생성
train[['영업장명', '메뉴명']] = train['영업장명_메뉴명'].str.split('_', expand=True)
# 요일 col 생성
train['weekday'] = train['영업일자'].dt.weekday.astype(int)

# 공휴일 col 생성
holidays["locdate"] = pd.to_datetime(holidays["locdate"])
holiday_dates = set(holidays['locdate'])
train['is_holiday'] = train['영업일자'].isin(holiday_dates).astype(int)

train['is_weekend'] = train['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)
# 5) isSandwich: 오늘은 평일(0)이고, 어제/내일이 모두 쉬는 날(1)인 경우 1
train["non_work"] = ((train["is_holiday"] == 1) | (train["is_weekend"] == 1)).astype(int)

train["is_sandwich"] = 0
train.loc[
    (train["non_work"] == 0) &
    (train["non_work"].shift(1) == 1) &
    (train["non_work"].shift(-1) == 1),
    "is_sandwich"
] = 1


In [346]:
# 1) 요일 컬럼 만들기
train['weekday'] = pd.to_datetime(train['영업일자']).dt.weekday  # 0=월, 6=일

# 2) 메뉴별 요일별 평균 매출 계산
menu_dow_avg = (
    train.groupby(['영업장명_메뉴명','weekday'], as_index=False)['매출수량']
         .mean()
         .rename(columns={'매출수량':'요일평균매출'})
)

# 3) 메뉴별 min–max 스케일링 (요일 단위)
g = menu_dow_avg.groupby('영업장명_메뉴명')['요일평균매출']
menu_dow_avg['weekday_score'] = (menu_dow_avg['요일평균매출'] - g.transform('min')) / \
                             (g.transform('max') - g.transform('min'))

# max=min → 모든 요일 평균이 동일 → 1로 통일
menu_dow_avg.loc[g.transform('max') == g.transform('min'), 'weekday_score'] = 1.0

# 4) 원본 데이터에 merge
train = train.merge(
    menu_dow_avg[['영업장명_메뉴명','weekday','weekday_score']],
    on=['영업장명_메뉴명','weekday'],
    how='left'
)

In [347]:
# 0/1로 되어 있다고 가정: non_work = (주말 or 휴일) 1, 평일 0
train['영업일자'] = pd.to_datetime(train['영업일자'])

# 1) 날짜 단위 non_work(하루에 여러 행이면 max로 대표)
daily = (train.groupby('영업일자', as_index=False)['non_work']
               .max()
               .sort_values('영업일자'))

# 2) 날짜 기준으로 before/after 플래그 계산
daily['is_before_holiday'] = (
    (daily['non_work'].eq(0)) &
    (daily['non_work'].shift(-1, fill_value=0).eq(1))
).astype(int)

daily['is_after_holiday'] = (
    (daily['non_work'].eq(0)) &
    (daily['non_work'].shift(1, fill_value=0).eq(1))
).astype(int)

# 3) 원본 train에 날짜로 merge
train = train.merge(
    daily[['영업일자', 'is_before_holiday', 'is_after_holiday']],
    on='영업일자', how='left'
)

# 필요하면 non_work 컬럼 정리
# train = train.drop(columns=['non_work'], errors='ignore')

# Feature about top menu --todo--

In [348]:
# 예시: train 데이터에 영업장명, 메뉴명, 매출수량 컬럼이 있다고 가정
# train = pd.read_csv("train.csv")

# 1) 영업장별 메뉴별 총매출
menu_sales = (
    train.groupby(['영업장명', '메뉴명'], as_index=False)['매출수량']
         .sum()
         .rename(columns={'매출수량': '총매출수량'})
)

# 2) 영업장 안에서 순위(내림차순: 많이 팔린 메뉴가 1위)
menu_sales['rank'] = menu_sales.groupby('영업장명')['총매출수량'] \
                               .rank(method='average', ascending=False)

# 3) 영업장별 메뉴 개수
menu_sales['n'] = menu_sales.groupby('영업장명')['총매출수량'].transform('size')

# 4) 0~1 스케일: (n - rank) / (n - 1)
#    → rank=1이면 1, rank=n이면 0 (n=1인 경우 예외 처리)
menu_sales['rank01'] = (menu_sales['n'] - menu_sales['rank']) / (menu_sales['n'] - 1)
menu_sales.loc[menu_sales['n'] == 1, 'rank01'] = 1.0  # 해당 영업장에 메뉴가 1개뿐이면 1

# 5) 원본 train에 붙이기
train = train.merge(
    menu_sales[['영업장명', '메뉴명', 'rank01']], 
    on=['영업장명', '메뉴명'], how='left'
).rename(columns={'rank01': 'menu_rank'})

# menu_ category

In [349]:
menu_category = {
    '1인 수저세트': '기타',
    'BBQ55(단체)': '메인메뉴',
    '대여료 60,000원': '기타',
    '대여료 30,000원': '기타',
       '대여료 90,000원': '기타',
       '본삼겹 (단품,실내)': '메인메뉴',
       '스프라이트 (단체)': '음료',
       '신라면': '추가메뉴',
       '쌈야채세트': '추가메뉴',
       '쌈장': '추가메뉴',
       '육개장 사발면': '추가메뉴',
       '일회용 소주컵': '기타',
       '일회용 종이컵': '기타',
       '잔디그늘집 대여료 (12인석)': '기타',
       '잔디그늘집 대여료 (6인석)': '기타',
       '잔디그늘집 의자 추가': '기타',
       '참이슬 (단체)': '주류',
       '친환경 접시 14cm': '기타',
       '친환경 접시 23cm': '기타',
       '카스 병(단체)': '주류',
       '콜라 (단체)': '음료',
       '햇반': '추가메뉴',
       '허브솔트': '추가메뉴',
       '(단체) 공깃밥': '추가메뉴',
       '(단체) 생목살 김치전골 2.0': '메인메뉴',
       '(단체) 은이버섯 갈비탕': '메인메뉴',
       '(단체) 한우 우거지 국밥': '메인메뉴',
       '(단체) 황태해장국 3/27까지': '메인메뉴',
       '(정식) 된장찌개': '메인메뉴',
       '(정식) 물냉면 ': '메인메뉴',
       '(정식) 비빔냉면': '메인메뉴',
       '(후식) 된장찌개': '추가메뉴',
       '(후식) 물냉면': '추가메뉴',
       '(후식) 비빔냉면': '추가메뉴',
       '갑오징어 비빔밥': '메인메뉴',
       '갱시기': '메인메뉴',
       '공깃밥': '추가메뉴',
       '꼬막 비빔밥': '메인메뉴',
       '느린마을 막걸리': '주류',
       '담하 한우 불고기': '메인메뉴',
       '담하 한우 불고기 정식': '메인메뉴',
       '더덕 한우 지짐': '메인메뉴',
       '들깨 양지탕': '메인메뉴',
       '라면사리': '추가메뉴',
       '룸 이용료': '기타',
       '메밀면 사리': '추가메뉴',
       '명인안동소주': '주류',
       '명태회 비빔냉면': '메인메뉴',
       '문막 복분자 칵테일': '주류',
       '봉평메밀 물냉면': '메인메뉴',
       '생목살 김치찌개': '메인메뉴',
       '스프라이트': '음료',
       '은이버섯 갈비탕': '메인메뉴',
       '제로콜라': '음료',
       '참이슬': '주류',
       '처음처럼': '주류',
       '카스': '주류',
       '콜라': '음료',
       '테라': '주류',
       '하동 매실 칵테일': '주류',
       '한우 떡갈비 정식': '메인메뉴',
       '한우 미역국 정식': '메인메뉴',
       '한우 우거지 국밥': '메인메뉴',
       '한우 차돌박이 된장찌개': '메인메뉴',
       '황태해장국': '메인메뉴',
       'AUS (200g)': '메인메뉴',
       'G-Charge(3)': '기타',
       'Gls.Sileni': '주류',
       'Gls.미션 서드': '주류',
       'Open Food': '기타',
       '그릴드 비프 샐러드': '메인메뉴',
       '까르보나라': '메인메뉴',
       '모둠 해산물 플래터': '메인메뉴',
       '미션 서드 카베르네 쉬라': '메인메뉴',
       '버섯 크림 리조또': '메인메뉴',
       '빵 추가 (1인)': '추가메뉴',
       '시저 샐러드 ': '메인메뉴',
       '아메리카노': '음료',
       '알리오 에 올리오 ': '메인메뉴',
       '양갈비 (4ps)': '메인메뉴',
       '자몽리치에이드': '음료',
       '하이네켄(생)': '주류',
       '한우 (200g)': '메인메뉴',
       '해산물 토마토 리조또': '메인메뉴',
       '해산물 토마토 스튜 파스타': '메인메뉴',
       '해산물 토마토 스파게티': '메인메뉴',
       '(단체)브런치주중 36,000': '메인메뉴',
       '(오븐) 하와이안 쉬림프 피자': '메인메뉴',
       '(화덕) 불고기 페퍼로니 반반피자': '메인메뉴',
       'BBQ Platter': '메인메뉴',
       'BBQ 고기추가': '추가메뉴',
       '글라스와인 (레드)': '주류',
       '레인보우칵테일(알코올)': '주류',
       '미라시아 브런치 (패키지)': '메인메뉴',
       '버드와이저(무제한)': '주류',
       '보일링 랍스타 플래터': '메인메뉴',
       '보일링 랍스타 플래터(덜매운맛)': '메인메뉴',
       '브런치 2인 패키지 ': '메인메뉴',
       '브런치 4인 패키지 ': '메인메뉴',
       '브런치(대인) 주말': '메인메뉴',
       '브런치(대인) 주중': '메인메뉴',
       '브런치(어린이)': '메인메뉴',
       '쉬림프 투움바 파스타': '메인메뉴',
       '스텔라(무제한)': '주류',
       '애플망고 에이드': '음료',
       '얼그레이 하이볼': '주류',
       '오븐구이 윙과 킬바사소세지': '메인메뉴',
       '유자 하이볼': '주류',
       '잭 애플 토닉': '주류',
       '칠리 치즈 프라이': '추가메뉴',
       '코카콜라': '음료',
       '코카콜라(제로)': '음료',
       '콥 샐러드': '추가메뉴',
       '파스타면 추가(150g)': '추가메뉴',
       '핑크레몬에이드': '음료',
       'Cass Beer': '주류',
       'Conference L1': '연회장 대여',
       'Conference L2': '연회장 대여',
       'Conference L3': '연회장 대여',
       'Conference M1': '연회장 대여',
       'Conference M8': '연회장 대여',
       'Conference M9': '연회장 대여',
       'Convention Hall': '연회장 대여',
       'Cookie Platter': '디저트',
       'Grand Ballroom': '연회장 대여',
       'OPUS 2': '연회장 대여',
       'Regular Coffee': '음료',
       '골뱅이무침': '메인메뉴',
       '돈목살 김치찌개 (밥포함)': '메인메뉴',
       '로제 치즈떡볶이': '메인메뉴',
       '마라샹궈': '메인메뉴',
       '매콤 무뼈닭발&계란찜': '메인메뉴',
       '모둠 돈육구이(3인)': '메인메뉴',
       '삼겹살추가 (200g)': '추가메뉴',
       '야채추가': '추가메뉴',
       '왕갈비치킨': '메인메뉴',
       '주먹밥 (2ea)': '추가메뉴',
       '공깃밥(추가)': '추가메뉴',
       '구슬아이스크림': '디저트',
       '단체식 13000(신)': '메인메뉴',
       '단체식 18000(신)': '메인메뉴',
       '돼지고기 김치찌개': '메인메뉴',
       '복숭아 아이스티': '음료',
       '새우 볶음밥': '메인메뉴',
       '새우튀김 우동': '메인메뉴',
       '샷 추가': '추가메뉴',
       '수제 등심 돈까스': '메인메뉴',
       '아메리카노(HOT)': '음료',
       '아메리카노(ICE)': '음료',
       '약 고추장 돌솥비빔밥': '메인메뉴',
       '어린이 돈까스': '메인메뉴',
       '오픈푸드': '기타',
       '진사골 설렁탕': '메인메뉴',
       '짜장면': '메인메뉴',
       '짜장밥': '메인메뉴',
       '짬뽕': '메인메뉴',
       '짬뽕밥': '메인메뉴',
       '치즈돈까스': '메인메뉴',
       '카페라떼(HOT)': '음료',
       '카페라떼(ICE)': '음료',
       '한상 삼겹구이 정식(2인) 소요시간 약 15~20분': '메인메뉴',
       '꼬치어묵': '메인메뉴',
       '떡볶이': '메인메뉴',
       '생수': '음료',
       '치즈 핫도그': '디저트',
       '페스츄리 소시지': '디저트',
       '단호박 식혜 ': '음료',
       '병천순대': '메인메뉴',
       '참살이 막걸리': '주류',
       '찹쌀식혜': '음료',
       '해물파전': '메인메뉴',
       '메밀미숫가루': '음료',
       '아메리카노 HOT': '음료',
       '아메리카노 ICE': '음료',
       '카페라떼 ICE': '음료',
       '현미뻥스크림': '디저트'

}


In [350]:
# Assign the menu_category to df
train['menu_category'] = train['메뉴명'].map(menu_category)
train['menu_category'], uniques = pd.factorize(train['menu_category'])

# avg_sales_all_days

In [351]:
menu_avg = train.groupby('메뉴명')['매출수량'].mean()
train['avg_sales_all_days'] = train['메뉴명'].map(menu_avg)

# Feature about zero
1. avg_sales_nonzero_days
2. zero_sales_day_ratio
3. is_sparse_menu

In [352]:
nonzero_avg = train[train['매출수량'] > 0].groupby('메뉴명')['매출수량'].mean()
# Assign the nonzero_avg to df
train['avg_sales_nonzero_days'] = train['메뉴명'].map(nonzero_avg)
# 매출수량 > 0인 경우만 메뉴별 분산 계산
nonzero_var = (
    train[train['매출수량'] > 0]
    .groupby('메뉴명')['매출수량']
    .var()   # ← 분산
)

# train에 새로운 칼럼으로 추가
train['var_sales_nonzero_days'] = train['메뉴명'].map(nonzero_var)

In [353]:
zero_ratio = train.groupby('메뉴명')['매출수량'].apply(lambda x: (x.eq(0).sum() / len(x)) * 100)

# Assign the zero_ratio to df
train['zero_sales_day_ratio'] = train['메뉴명'].map(zero_ratio)

In [354]:
# train['is_sparse_menu'] = np.where(train['zero_sales_day_ratio'] > 50, 1, 0)

# Feature about 연회장
1. banquet_type

In [355]:
df_연회장 = train[train['영업장명']=='연회장'].pivot_table(index='영업일자',columns='메뉴명',values='매출수량', aggfunc = 'sum').reset_index()

In [356]:
df_연회장['연회장 대여'] = df_연회장[['Conference L1','Conference L2','Conference L3','Conference M1','Conference M8','Conference M9','Convention Hall','Grand Ballroom','OPUS 2']].sum(axis=1)
df_연회장['음료 및 쿠키'] = df_연회장[['Cookie Platter','Cass Beer','Regular Coffee']].sum(axis=1)
df_연회장['음식'] = df_연회장[['골뱅이무침','공깃밥','돈목살 김치찌개 (밥포함)','로제 치즈떡볶이','마라샹궈','매콤 무뼈닭발&계란찜','모둠 돈육구이(3인)','삼겹살추가 (200g)','야채추가','왕갈비치킨','주먹밥 (2ea)']].sum(axis=1)

In [357]:
def banquet_type(row):
    if row['연회장 대여'] > 0 and row['음식'] == 0 and row['음료 및 쿠키'] == 0:
        return 1 # 대여만
    elif row['연회장 대여'] == 0 and row['음식'] == 0 and row['음료 및 쿠키'] > 0:
        return 2 # 음료및쿠키만
    elif row['연회장 대여'] == 0 and row['음식'] > 0 and row['음료 및 쿠키'] == 0:
        return 3 # 음식만
    elif row['연회장 대여'] > 0 and row['음식'] == 0 and row['음료 및 쿠키'] > 0:
        return 4 # 대여+음료및쿠키
    elif row['연회장 대여'] > 0 and row['음식'] > 0 and row['음료 및 쿠키'] == 0:
        return 5 # 대여+음식
    elif row['연회장 대여'] == 0 and row['음식'] > 0 and row['음료 및 쿠키'] > 0:
        return 6 # 음식+음료및쿠키
    elif row['연회장 대여'] > 0 and row['음식'] > 0 and row['음료 및 쿠키'] > 0:
        return 7 # 대여+음료및쿠키+음식
    else:
        return 0 # 연회장 총매출이 0인경우
df_연회장['banquet_type'] = df_연회장.apply(banquet_type, axis=1)


# Features
1. is_drink
2. is_alcohol
3. is_set_menu

In [358]:
# drink_keywords = ['콜라', '스프라이트', '제로콜라', '자몽리치에이드', '애플망고 에이드', '핑크레몬에이드', '아메리카노',
#                   '식혜', '메밀미숫가루', '아메리카노', '카페라떼', '복숭아 아이스티','샷 추가',
#                   '생수']

# alcohol_keywords = ['Gls.Sileni', 'Gls.미션 서드', '미션 서드 카메르네 쉬라', '하이네켄', '막걸리',
#                     '와인', '버드와이저', '스텔라', '하이볼', '잭 애플 토닉', '참이슬', '소주', '처음처럼',
#                     '카스', '테라', '칵테일', 'Cass']

# set_keywords = ['정식']

# train['is_drink'] = train['영업장명_메뉴명'].apply(
#     lambda x: 1 if any(keyword in str(x) for keyword in drink_keywords) else 0
# )

# train['is_alcohol'] = train['영업장명_메뉴명'].apply(
#     lambda x: 1 if (
#         any(keyword in str(x) for keyword in alcohol_keywords)
#         and '컵' not in str(x)
#     ) else 0
# )

# train['is_set_menu'] = train['영업장명_메뉴명'].apply(
#     lambda x: 1 if (
#         any(keyword in str(x) for keyword in set_keywords)
#     ) else 0
# )

# #매출수량이 문자열이면 숫자로 변환
# train['매출수량'] = pd.to_numeric(train['매출수량'], errors='coerce')


# demand_volatility, demand_stability

In [359]:
menu_stats=(
    train.groupby('영업장명_메뉴명')['매출수량']
    .agg(['mean','std'])
    .reset_index()
)

menu_stats['demand_volatility']=menu_stats['std']/menu_stats['mean']
# menu_stats['demand_stability']=1/menu_stats['demand_volatility']

menu_stats.rename(columns={'mean':'평균매출수량','std':'표준편차'},inplace=True)

# Merge menu_stats back to df
train=train.merge(menu_stats[['영업장명_메뉴명', 'demand_volatility']], on='영업장명_메뉴명',how='left')

# Add New

# **데이터 전처리**

1. **is_spike**: 전식당, 변할 수 있는 가능성을 학습하게끔 하는 것, 당일 수치가 최근 7일간 평균 + 2 × 표준편차보다 크면 1, 아니면 0, 위로 갑자기 튀는 것을 포착
2. **is_drop**: 전식당, 변할 수 있는 가능성을 학습하게끔 하는 것, 당일 수치가 최근 7일간 평균 − 2 × 표준편차보다 작으면 1, 아니면 0, 아래로 갑자기 튀는 것을 포착
3. **is_weekday_price**: 미라시아, 요금제가 주중 기준인지 여부 구분	메뉴명에 '주중'이 포함되면 1, 아니면 0, 주중 요금이 적용된 메뉴인지 여부
4. **is_weekend_price**: 미라시아, 요금제가 주말 기준인지 여부 구분	메뉴명에 '주말'이 포함되면 1, 아니면 0, 주말 요금이 적용된 메뉴인지 여부
5. **seasonal_index**: 전식당,

    • 1분기 (Q1): 1월 1일부터 3월 31일까지

    • 2분기 (Q2): 4월 1일부터 6월 30일까지

    • 3분기 (Q3): 7월 1일부터 9월 30일까지

    • 4분기 (Q4): 10월 1일부터 12월 31일까지

    월별 또는 분기별 매출 패턴 분석하여 생성, 분기별 매출 수치화
6. **미라시아 단체 관련 변수( brunch_flag, hallroom_flag)** :
    
    6-1. **brunch_flag**: 단체 브런치 메뉴 매출이 생긴 날의 플래그, 연회장_룸타입에만 플래그를 세운다.

    6-2. **hallroom_flag**: 연회장_룸타입 매출이 생긴 날의 플래그, (단체)브런치주중 36,000 에만 플래그를 세운다.

# quarter_index(all)

생성 목적: 분기별 매출 수치화

생성 방법:

• 1분기 (Q1): 1월 1일부터 3월 31일까지

• 2분기 (Q2): 4월 1일부터 6월 30일까지

• 3분기 (Q3): 7월 1일부터 9월 30일까지

• 4분기 (Q4): 10월 1일부터 12월 31일까지

분기별 매출 패턴 분석하여 누적합으로 생성

In [360]:
def attach_solar_term(
    train: pd.DataFrame,
    terms: pd.DataFrame,
    date_col: str = "영업일자",
    term_date_col: str = "locdate",
    term_name_col: str = "solar_term",
    out_col: str = "solar_term"
) -> pd.DataFrame:
    df = train.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    terms = terms.copy()
    terms[term_date_col] = pd.to_datetime(terms[term_date_col], errors='coerce')

    # 절기 구간 만들기
    terms = terms.sort_values(term_date_col).reset_index(drop=True)
    terms['end_date'] = terms[term_date_col].shift(-1) - pd.Timedelta(days=1)
    terms.loc[terms.index[-1], 'end_date'] = df[date_col].max()

    # ★ 절기명 컬럼을 임시 이름으로 바꿔서 머지
    tmp_col = "_term_name_tmp"
    right = terms[[term_date_col, term_name_col, 'end_date']].rename(columns={term_name_col: tmp_col})

    merged = pd.merge_asof(
        df.sort_values(date_col),
        right.sort_values(term_date_col),
        left_on=date_col,
        right_on=term_date_col,
        direction='backward'
    )

    in_range = merged[date_col] <= merged['end_date']
    merged[out_col] = merged[tmp_col].where(in_range)

    # 임시 컬럼만 삭제 (out_col은 유지)
    merged = merged.drop(columns=[term_date_col, 'end_date', tmp_col]).sort_index()
    return merged

In [361]:
# # 0) 날짜 보정
# if df['영업일자'].dtype == 'O':
#     df['영업일자'] = pd.to_datetime(df['영업일자'], errors='coerce')

# 1) '분기' → 'quarter'로 변경 (없으면 새로 생성)
#     Q1, Q2, Q3, Q4 형식으로 생성
def get_quarter_code(m):
        if m in (1,2,3): return 0
        if m in (4,5,6): return 1
        if m in (7,8,9): return 2
        return 3
train['quarter'] = train['month'].apply(get_quarter_code)
train['season'] = train.apply(
    lambda x: 0 if x['month'] in [3, 4, 5] else
              (1 if x['month'] in [6, 7, 8] else
               (2 if x['month'] in [9, 10, 11] else 3)),
    axis=1
)

# terms 예시: columns=['locdate','절기'] 라면 term_name_col='절기'로 지정
train = attach_solar_term(
    train,
    terms,                   # 절기 lookup DF
    date_col="영업일자",
    term_date_col="locdate",
    term_name_col="solar_term",    # 실제 컬럼명 맞추기!
    out_col="solar_term"     # train에 새로 생길 이름
)


In [362]:
def cumsum(train, col, newcol):    
# 0) 날짜 보정
    if train['영업일자'].dtype == 'O':
        train['영업일자'] = pd.to_datetime(train['영업일자'], errors='coerce')

    # 2) 임시로 매장/메뉴 분리(컬럼 추가 X)
    keys = train['영업장명_메뉴명'].str.split('_', n=1, expand=True)
    g_store, g_menu = keys[0], keys[1]

    # 3) 그룹별 전일까지 누적합 → 전체 열에 한 번에 대입 (정수 고정)
    #    - 정렬은 누적 순서만 위해 잠깐 사용, 결과는 원래 인덱스로 돌아옴
    train_sorted = train.sort_values([col, '영업일자']).copy()
    seasonal_series = (
        train_sorted
        .groupby([g_store.reindex(train_sorted.index),
                    g_menu.reindex(train_sorted.index),
                    train_sorted[col]], sort=False)['매출수량']
        .transform(lambda s: s.shift(1).cumsum())
    )

    train[newcol] = (
        seasonal_series.reindex(train_sorted.index)
                    .reindex(train.index)
                    .fillna(0)
                    .astype('int64')
    )
    return train
train = cumsum(train, 'quarter', 'quarter_sum')
train = cumsum(train, 'season', 'season_sum')
train = cumsum(train, 'solar_term', 'solar_term_sum')


# **미라시아 단체 관련 변수( brunch_flag, hallroom_flag)**

**brunch_flag**: 단체 브런치 메뉴 매출이 생긴 날의 플래그, 연회장_룸타입('Grand Ballroom', 'Convention Hall', 'Conference L', 'Conference M', 'OPUS 2')에만 플래그를 세운다.

**hallroom_flag**: 연회장_룸타입 매출이 생긴 날의 플래그, (단체)브런치주중 36,000 에만 플래그를 세운다.

In [363]:
TARGET = '미라시아_(단체)브런치주중 36,000'
HALL_ROOMS = {'Grand Ballroom', 'Convention Hall', 'Conference L', 'Conference M', 'OPUS'}

# 날짜형 변환
if train['영업일자'].dtype == 'O':
    train['영업일자'] = pd.to_datetime(train['영업일자'], errors='coerce')

# '영업장명_메뉴명' 분리
tokens = train['영업장명_메뉴명'].str.split('_', n=1, expand=True)
store0 = tokens[0].astype(str).str.strip()   # 예: '연회장', '미라시아', ...
store1 = tokens[1].astype(str).str.strip()   # 예: 'Grand Ballroom', '(단체)브런치주중 36,000', ...

# 1) 연회장 매출 발생 날짜
hall_mask = (store0.eq('연회장')) & (store1.isin(HALL_ROOMS)) & (train['매출수량'] > 0)
hall_dates = train.loc[hall_mask, '영업일자'].unique()

# 2) 브런치 매출 발생 날짜
brunch_mask = train['영업장명_메뉴명'].eq(TARGET) & (train['매출수량'] > 0)
brunch_dates = train.loc[brunch_mask, '영업일자'].unique()

# 3) 플래그 생성 (반대로 반영)
# 초기화: 전부 0
train['brunch_flag'] = 0      # ← 연회장 라인에 찍힘 (브런치 매출 발생일 기준)
train['hallroom_flag'] = 0    # ← 미라시아 단체 브런치 라인에 찍힘 (연회장 매출 발생일 기준)

# A) 단체 브런치 매출 발생일 → "연회장_*" 행에 brunch_flag=1
train.loc[hall_mask & train['영업일자'].isin(brunch_dates), 'brunch_flag'] = 1

# B) 연회장 매출 발생일 → "미라시아_(단체)브런치주중 36,000" 행에 hallroom_flag=1
target_row_mask = train['영업장명_메뉴명'].eq(TARGET)
train.loc[target_row_mask & train['영업일자'].isin(hall_dates), 'hallroom_flag'] = 1

In [364]:
# Add features from the 'train' DataFrame to 'df'
# Ensure '영업일자' is datetime in both dataframes for merging
train['영업일자'] = pd.to_datetime(train['영업일자'], errors='coerce')

# Merge based on '영업일자' and '영업장명_메뉴명'
# Recreate '영업장명_메뉴명' in train for merging if it was dropped
if '영업장명_메뉴명' not in train.columns:
    train['영업장명_메뉴명'] = train['영업장명'] + '_' + train['메뉴명']

# Add banquet_type from df_연회장
# Ensure '영업일자' is datetime in df_연회장
df_연회장['영업일자'] = pd.to_datetime(df_연회장['영업일자'], errors='coerce')

train = pd.merge(train, df_연회장[['영업일자', 'banquet_type']], on='영업일자', how='left')

# Fill NaN values in banquet_type with 0 (assuming 0 means no banquet)
train['banquet_type'] = train['banquet_type'].fillna(0).astype(int)

# Store trian.csv

In [365]:
# train.drop(columns=['영업장명','메뉴명'], errors='ignore', inplace=True)
train[['영업장명', '메뉴명']] = train['영업장명_메뉴명'].str.split('_', expand=True)
# train.drop(columns=['day_of_quarter'], inplace=True, errors='ignore')
# train.drop(columns=['cum_share_ref'], inplace=True, errors='ignore')
# train DataFrame을 CSV로 저장 (모든 feature 포함)
train = train.sort_values(['영업장명_메뉴명', '영업일자']).reset_index(drop=True)
train.to_csv("re_train_06.csv", index=False, encoding="utf-8-sig")

print(train.columns)

print("train DataFrame이 output.csv로 저장되었습니다.")

Index(['영업일자', '영업장명_메뉴명', '매출수량', 'time_idx', 'year', 'month', 'day', '영업장명',
       '메뉴명', 'weekday', 'is_holiday', 'is_weekend', 'non_work', 'is_sandwich',
       'weekday_score', 'is_before_holiday', 'is_after_holiday', 'menu_rank',
       'menu_category', 'avg_sales_all_days', 'avg_sales_nonzero_days',
       'var_sales_nonzero_days', 'zero_sales_day_ratio', 'demand_volatility',
       'quarter', 'season', 'solar_term', 'quarter_sum', 'season_sum',
       'solar_term_sum', 'brunch_flag', 'hallroom_flag', 'banquet_type'],
      dtype='object')
train DataFrame이 output.csv로 저장되었습니다.


# Task
Apply the same feature engineering steps (date features, holiday features, spike/drop, seasonal index, banquet type, etc.) that were applied to the training data to the following test files: "TEST_01.csv", "TEST_02.csv", "TEST_03.csv", "TEST_04.csv", "TEST_05.csv", "TEST_06.csv", "TEST_07.csv", "TEST_08.csv", "TEST_09.csv". Ensure that all engineered features are added as new columns to the respective DataFrames loaded from these files.

## Identify test files

### Subtask:
Create a list of all the test file paths (`TEST_01.csv` to `TEST_09.csv`).


**Reasoning**:
Create a list containing the file paths for the test datasets.



In [366]:
test_files = [f"TEST_{i:02d}.csv" for i in range(0, 10)] # Changed range from 1 to 0 to include TEST_00
print(test_files)

['TEST_00.csv', 'TEST_01.csv', 'TEST_02.csv', 'TEST_03.csv', 'TEST_04.csv', 'TEST_05.csv', 'TEST_06.csv', 'TEST_07.csv', 'TEST_08.csv', 'TEST_09.csv']


## Define feature engineering function

### Subtask:
Create a function that takes a DataFrame (like the one loaded from a test file) and applies all the necessary feature engineering steps (date features, holiday features, spike/drop, seasonal index, banquet type, etc.) to it, returning the processed DataFrame.


**Reasoning**:
Define a function `engineer_features` that takes a DataFrame and applies all the feature engineering steps. This function will include date features, holiday features, spike/drop detection, weekday/weekend price flags, seasonal index, brunch/hallroom flags, and banquet type merging.



In [ ]:
def engineer_features(df_test, holiday_df, banquet_df, terms_df=None, terms_csv_path=None):
    """Applies feature engineering steps to a test DataFrame.
       terms_df: (선택) 절기 테이블 DataFrame (cols: ['locdate','solar_term'])
       terms_csv_path: (선택) terms_df가 없을 때 읽어올 CSV 경로
    """
    import pandas as pd
    df_test[['영업장명', '메뉴명']] = df_test['영업장명_메뉴명'].str.split('_', expand=True)


    # --- 기본 날짜 피처 ---
    df_test = df_test.copy()
    df_test['영업일자'] = pd.to_datetime(df_test['영업일자'], errors='coerce')
    df_test['time_idx'] = (df_test['영업일자'] - df_test['영업일자'].min()).dt.days
    df_test['year']    = df_test['영업일자'].dt.year.astype(int)
    df_test['month']   = df_test['영업일자'].dt.month.astype(int)
    df_test['day']     = df_test['영업일자'].dt.day.astype(int)
    df_test['weekday'] = df_test['영업일자'].dt.weekday.astype(int)

    # --- season 코드 (이미 사용 중) ---
    def get_season_code(m):
        if m in (12,1,2): return 0
        if m in (3,4,5):  return 1
        if m in (6,7,8):  return 2
        return 3
    df_test['season'] = df_test['month'].apply(get_season_code)
    def get_quarter_code(m):
        if m in (1,2,3): return 0
        if m in (4,5,6): return 1
        if m in (7,8,9): return 2
        return 3
    df_test['quarter'] = df_test['month'].apply(get_quarter_code)
    df_test = attach_solar_term(
        df_test,
        terms,                   # 절기 lookup DF
        date_col="영업일자",
        term_date_col="locdate",
        term_name_col="solar_term",    # 실제 컬럼명 맞추기!
        out_col="solar_term"     # train에 새로 생길 이름
    )

    def attach_avg_by_keys(train, df_test, value_col, out_col=None,
                        keys=('영업장명_메뉴명','month','day')):
        # train에서 키별 평균 테이블
        mp = (train.groupby(list(keys), as_index=False)[value_col]
                    .mean()
                    .rename(columns={value_col: out_col or value_col}))
        # test에 merge
        df_test = df_test.merge(mp, on=list(keys), how='left')
        return df_test

    # 사용 예시: 각 *_sum 컬럼의 평균을 동일 키로 test에 붙이기
    df_test = attach_avg_by_keys(train, df_test, 'quarter_sum')
    df_test = attach_avg_by_keys(train, df_test, 'season_sum')
    df_test = attach_avg_by_keys(train, df_test, 'solar_term_sum')

    # 2. Holiday features
    holiday_df['locdate'] = pd.to_datetime(holiday_df['locdate'])
    df_test = pd.merge(
        df_test,
        holiday_df[['locdate', 'isHoliday']],
        how='left',
        left_on='영업일자',
        right_on='locdate'
    )
    df_test['is_holiday'] = df_test['isHoliday'].fillna('N').apply(lambda x: 1 if x == 'Y' else 0)
    df_test = df_test.drop(['locdate', 'isHoliday'], axis=1)  # 둘 다 삭제
    df_test['is_weekend'] = df_test['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)

    df_test["non_work"] = ((df_test["is_holiday"] == 1) | (df_test["is_weekend"] == 1)).astype(int)

    # train에서 menu_rank 매핑 테이블 추출 (중복 제거)
    menu_rank_map = train[['영업장명_메뉴명', 'menu_rank']].drop_duplicates()

    # df_test에 merge
    df_test = df_test.merge(
        menu_rank_map,
        on='영업장명_메뉴명',
        how='left'
    )

    # 0/1로 정의: non_work = (주말 or 휴일) 이면 1, 평일이면 0
    df_test['영업일자'] = pd.to_datetime(df_test['영업일자'])

    # 1) 날짜 단위 대표 non_work 만들기 (하루에 여러 행이면 max로 대표)
    daily = (df_test.groupby('영업일자', as_index=False)['non_work']
                .max()
                .sort_values('영업일자'))

    # 2) 샌드위치 데이: 오늘은 근무일(0)이고, 전/다음날은 쉬는날(1)
    daily['is_sandwich'] = (
        (daily['non_work'].eq(0)) &
        (daily['non_work'].shift(1,  fill_value=0).eq(1)) &
        (daily['non_work'].shift(-1, fill_value=0).eq(1))
    ).astype(int)

    # 3) 원본 df_test에 날짜로 merge (기존 컬럼 있으면 덮어쓰기)
    df_test = df_test.drop(columns=['is_sandwich'], errors='ignore') \
                    .merge(daily[['영업일자','is_sandwich']], on='영업일자', how='left')
    # 0/1로 되어 있다고 가정: non_work = (주말 or 휴일) 1, 평일 0
    df_test['영업일자'] = pd.to_datetime(df_test['영업일자'])

    # 1) 날짜 단위 non_work(하루에 여러 행이면 max로 대표)
    daily = (df_test.groupby('영업일자', as_index=False)['non_work']
                .max()
                .sort_values('영업일자'))

    # 2) 날짜 기준으로 before/after 플래그 계산
    daily['is_before_holiday'] = (
        (daily['non_work'].eq(0)) &
        (daily['non_work'].shift(-1, fill_value=0).eq(1))
    ).astype(int)

    daily['is_after_holiday'] = (
        (daily['non_work'].eq(0)) &
        (daily['non_work'].shift(1, fill_value=0).eq(1))
    ).astype(int)

    # 3) 원본 df_test에 날짜로 merge
    df_test = df_test.merge(
        daily[['영업일자', 'is_before_holiday', 'is_after_holiday']],
        on='영업일자', how='left'
    )

# 필요하면 non_work 컬럼 정리
# train = train.drop(columns=['non_work'], errors='ignore')
    df_test = df_test.drop(columns=['non_work'], errors='ignore')

    # train에서 필요한 칼럼만 매핑 테이블로 추출 (중복 제거)
    weekday_map = train[['영업장명_메뉴명', 'weekday', 'weekday_score']].drop_duplicates()

    # merge로 붙이기
    df_test = df_test.merge(
        weekday_map,
        on=['영업장명_메뉴명', 'weekday'],
        how='left'
    )

    # 3. lookup features

    feat_cols = ['영업장명_메뉴명',
             'menu_category','avg_sales_all_days','avg_sales_nonzero_days',
             'zero_sales_day_ratio','demand_volatility', 'var_sales_nonzero_days']

    train_feats = (train[feat_cols]
               .groupby('영업장명_메뉴명', as_index=False)
               .agg('first'))  # 필요시 mean/max로 변경

    df_test = df_test.merge(train_feats, on='영업장명_메뉴명',
                        how='left', validate='many_to_one')

    # 6. Brunch/Hallroom flags
    TARGET = '미라시아_(단체)브런치주중 36,000'
    HALL_ROOMS = {'Grand Ballroom', 'Convention Hall', 'Conference L', 'Conference M', 'OPUS'}

    tokens = df_test['영업장명_메뉴명'].str.split('_', n=1, expand=True)
    store0 = tokens[0].astype(str).str.strip()
    store1 = tokens[1].astype(str).str.strip()

    # Recalculate hall_dates and brunch_dates from the *original train* data (assuming 'train' df is available globally)
    if 'train' in globals():
        train_tokens = train['영업장명_메뉴명'].str.split('_', n=1, expand=True)
        train_store0 = train_tokens[0].astype(str).str.strip()
        train_store1 = train_tokens[1].astype(str).str.strip()

        train_hall_mask = (train_store0.eq('연회장')) & (train_store1.isin(HALL_ROOMS)) & (train['매출수량'] > 0)
        hall_dates = train.loc[train_hall_mask, '영업일자'].unique()

        train_brunch_mask = train['영업장명_메뉴명'].eq(TARGET) & (train['매출수량'] > 0)
        brunch_dates = train.loc[train_brunch_mask, '영업일자'].unique()
    else:
        # Fallback or error handling if train data is not available
        print("Warning: 'train' DataFrame not found. Cannot calculate brunch_dates and hall_dates.")
        hall_dates = []
        brunch_dates = []


    if 'brunch_flag' not in df_test.columns:
        df_test['brunch_flag'] = 0
    if 'hallroom_flag' not in df_test.columns:
        df_test['hallroom_flag'] = 0

    test_hall_mask = (store0.eq('연회장')) & (store1.isin(HALL_ROOMS))
    test_target_row_mask = df_test['영업장명_메뉴명'].eq(TARGET)

    df_test.loc[test_hall_mask & df_test['영업일자'].isin(brunch_dates), 'brunch_flag'] = 1
    df_test.loc[test_target_row_mask & df_test['영업일자'].isin(hall_dates), 'hallroom_flag'] = 1


    # 7. Banquet type
    # Ensure '영업일자' is datetime in banquet_df
    banquet_df['영업일자'] = pd.to_datetime(banquet_df['영업일자'], errors='coerce')
    df_test = pd.merge(df_test, banquet_df[['영업일자', 'banquet_type']], on='영업일자', how='left')
    df_test['banquet_type'] = df_test['banquet_type'].fillna(0).astype(int)


    return df_test

## Process each test file

### Subtask:
Iterate through the list of test file paths. For each file:
    - Load the CSV into a DataFrame.
    - Apply the feature engineering function to the DataFrame.
    - Store the processed DataFrame (e.g., in a dictionary or list).


**Reasoning**:
Iterate through the test files, apply the feature engineering function to each, and store the results.



In [368]:
processed_test_dfs = {}

# Load necessary dataframes outside the loop
print(os.getcwd())  # Ensure the current working directory is set correctly
terms_df = pd.read_csv('../solar_term_2023_2025.csv')  # Load the terms CSV
holiday_df = pd.read_csv('../holidays_2023_2025.csv')
banquet_df_full = pd.read_csv('re_train_06.csv') # Load the processed train data

# Prepare the banquet_df for merging (only date and type)
banquet_df_for_merge = banquet_df_full[['영업일자', 'banquet_type']].drop_duplicates()

for file_path in test_files:
    print(f"Processing {file_path}...")
    # Load the test data
    df_test = pd.read_csv(f"../../data/test/{file_path}")

    # Apply feature engineering
    # Pass the necessary dataframes to the function
    processed_df = engineer_features(df_test, holiday_df, banquet_df_for_merge, terms_df)

    # Store the processed DataFrame
    processed_test_dfs[file_path] = processed_df
    print(f"Finished processing {file_path}.")

# Display the first few rows of one of the processed dataframes to verify
if processed_test_dfs:
    first_file = list(processed_test_dfs.keys())[0]
    print(f"\nSample of processed {first_file}:")
    print(processed_test_dfs[first_file].columns.tolist())

/Users/garden/Desktop/lgaimers/Hackaton/DataProcessing/re_data_processed


Processing TEST_00.csv...
Finished processing TEST_00.csv.
Processing TEST_01.csv...
Finished processing TEST_01.csv.
Processing TEST_02.csv...
Finished processing TEST_02.csv.
Processing TEST_03.csv...
Finished processing TEST_03.csv.
Processing TEST_04.csv...
Finished processing TEST_04.csv.
Processing TEST_05.csv...
Finished processing TEST_05.csv.
Processing TEST_06.csv...
Finished processing TEST_06.csv.
Processing TEST_07.csv...
Finished processing TEST_07.csv.
Processing TEST_08.csv...
Finished processing TEST_08.csv.
Processing TEST_09.csv...
Finished processing TEST_09.csv.

Sample of processed TEST_00.csv:
['영업일자', '영업장명_메뉴명', '매출수량', '영업장명', '메뉴명', 'time_idx', 'year', 'month', 'day', 'weekday', 'season', 'quarter', 'solar_term', 'quarter_sum', 'season_sum', 'solar_term_sum', 'is_holiday', 'is_weekend', 'menu_rank', 'is_sandwich', 'is_before_holiday', 'is_after_holiday', 'weekday_score', 'menu_category', 'avg_sales_all_days', 'avg_sales_nonzero_days', 'zero_sales_day_ratio', 

## Save processed test data (optional)

### Subtask:
Save each processed test DataFrame to a new CSV file.


**Reasoning**:
Iterate through the processed test dataframes and save each one to a CSV file with an added suffix.



In [369]:
import os

output_dir = "./re_test_processed_04/" # Save in the current directory

for filename, df_processed in processed_test_dfs.items():
    # Construct output filename by adding '_processed' before the extension
    base, ext = os.path.splitext(filename)
    output_filename = f"{base}_processed{ext}"
    output_path = os.path.join(output_dir, output_filename)

    # Save the DataFrame to CSV
    df_processed.to_csv(output_path, index=False, encoding="utf-8-sig")
    

    print(f"Saved processed {filename} to {output_path}")

Saved processed TEST_00.csv to ./re_test_processed_04/TEST_00_processed.csv
Saved processed TEST_01.csv to ./re_test_processed_04/TEST_01_processed.csv
Saved processed TEST_02.csv to ./re_test_processed_04/TEST_02_processed.csv
Saved processed TEST_03.csv to ./re_test_processed_04/TEST_03_processed.csv
Saved processed TEST_04.csv to ./re_test_processed_04/TEST_04_processed.csv
Saved processed TEST_05.csv to ./re_test_processed_04/TEST_05_processed.csv
Saved processed TEST_06.csv to ./re_test_processed_04/TEST_06_processed.csv
Saved processed TEST_07.csv to ./re_test_processed_04/TEST_07_processed.csv
Saved processed TEST_08.csv to ./re_test_processed_04/TEST_08_processed.csv
Saved processed TEST_09.csv to ./re_test_processed_04/TEST_09_processed.csv
